Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import matplotlib.pyplot as plt

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [2]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [3]:
print(len(DIST_MATRIX))
print(type(DIST_MATRIX))
print(DIST_MATRIX[0:2].shape)

46
<class 'numpy.ndarray'>
(2, 46)


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [4]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]                      ### we want the last element equal to the initial 
    assert set(tsp) == set(range(len(CITIES)))    
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

nx.find_cycle(G) in the NetworkX library is used to find a cycle in a graph G. A cycle in a graph is a path that starts and ends at the same node, with each edge and node being visited only once (except the start/end node)

In [5]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

## First Greedy Algorithm

In [6]:
visited = np.full(len(CITIES), False)

dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])    #### here we are finding the closest city 
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))


logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

## EVOLUTION ALGORITHM

Population: A group of potential solutions to the problem, typically represented as individuals or chromosomes. The population evolves over generations.

Selection: The process of choosing individuals from the population to create offspring. Common methods include:

Tournament Selection: Randomly selecting a subset of individuals and choosing the best among them.
Roulette Wheel Selection: Choosing individuals based on their fitness probability.
Rank-Based Selection: Selecting based on the rank order of fitness values.
Genetic Operators:

Crossover (Recombination): Combines parts of two parent solutions to create offspring. Crossover types include single-point, multi-point, and uniform crossover.
Mutation: Randomly alters an individual’s genes to maintain genetic diversity. Mutation can be bit-flip for binary encoding or Gaussian mutation for real numbers.
Fitness Function: Measures how well each individual solves the problem. Fitness determines an individual's probability of being selected for reproduction.

Replacement Strategy: Determines how to form the next generation, often by combining the parents and offspring and selecting the best individuals, or by replacing only a portion of the population.

Termination Condition: Defines when the algorithm should stop. Common conditions include reaching a maximum number of generations, a solution meeting a desired fitness threshold, or observing no improvement over several generations.

Candidate solution ® Individual -> Set of candidate solutions ® Population ->
-> Ability to solve the problem ® Fitness -> Sequence of steps ® Generations

punti da fare:

    - generare le soluzioni 
    

In [7]:
def GreedyAlgorithm(index_starting):
    global DIST_MATRIX
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = index_starting
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])    #### here we are finding the closest city 
        #logging.debug(
        #    f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
        #)
        visited[closest] = True
        city = closest
        tsp.append(int(city))


    #logging.debug(
    #    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
    #)
    tsp.append(tsp[0])

    cost = tsp_cost(tsp)
    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {cost:.2f}km")

    return (tsp, cost)    

INFO Roulette wheel:Define the Number of Samples Needed: To form a new generation, we often need the same number of individuals as the current population. So, if the population has 𝑁 individuals, we usually draw N samples to maintain a constant population size.

## PARENT SELECTION

In [8]:
import random

def tournament_Selection( tao=5 ):   # we can change dinamically tao
    global population
    tournament_random = random.sample(population, tao)

    tournament_random = sorted( tournament_random , key = lambda x : x[1])

    return tournament_random[0]

def roulette_wheel():
    global population 
    # Problems: • Small selection pressure in large population, in large pop the probabilities are squeeze
    # Not efficient in large population 
    fitness_value = [el[1] for el in population ]
    total= sum(fitness_value)

    probabilities = [fitness / total for fitness in fitness_value]

    selected_individuals = random.choices(population , weights=probabilities, k=len(population))

    return selected_individuals  #### list of selection parent 

    

def rank_Based_Selection( p_max = 0.4 , p_min = 0.1):
    global population 

    population = sorted( population, key = lambda x : x[1] )

    N = len(population)
    ranked_pop = [  ( index , population[index]  ) for index in range(population)   ]


    # linear
    prob_rank = [     (ranked_pop_element[0] ,  ( p_min + ((p_max - p_min )*( N -ranked_pop_element[0] )/( N -ranked_pop_element[0] ) ) ) )for ranked_pop_element in ranked_pop      ]           

    selected_individuals = random.choices(population , weights=prob_rank[1], k=len(population))

    return selected_individuals

## MUTATION ALGORITHM

- Swap mutation
- Scramble Mutation
- Insert mutation
- Inversion Mutation


In [65]:
def swap_mutation(solution):

    start_point= solution[0]
    combined_solution = solution[1:-1]
    index1, index2 = random.sample(range(len(combined_solution)), 2)
    combined_solution[index1], combined_solution[index2] = combined_solution[index2], combined_solution[index1]
    combined_solution.insert(0, start_point)
    combined_solution.append(start_point)

    return combined_solution

def insert_mutation(solution): 
    start_point= solution[0]
    combined_solution = solution[1:-1]
    index1, index2 = random.sample(range(len(combined_solution)), 2)
    value = combined_solution.pop(index2)
    combined_solution.insert(index1+1, value)
    combined_solution.insert(0, start_point)
    combined_solution.append(start_point)
    print(combined_solution)

    return combined_solution




[0, 1, 2, 3, 4, 5, 0]
[0, 2, 3, 4, 5, 1, 0]
[0, 2, 3, 4, 5, 1, 0]


## COMBINATION 

In [141]:
#Inver Over

def inver_over_crossover(parent1, parent2):
    start_point_1= parent1[0]
    combined_solution_1 = parent1[:-1]
    print(parent1)
    print(parent2)
    print(combined_solution_1)
    start_point_2= parent2[0]
    combined_solution_2 = parent2[:-1]
    print(combined_solution_2 , " prendere da qui")

    index1_1= random.sample(range(len(combined_solution_1)-1) , k=1).pop()
    value_1_1 = combined_solution_1[index1_1]         # we get consecutive values
    value_1_2 = combined_solution_1[index1_1+1]       #### from one vector i get 2 values

    print(index1_1 ,"element ", value_1_1 ,value_1_2)

    value_2_1 = None
    value_2_2 = None
    index2_1 = None
    index2_2 = None
    for el in range(len(combined_solution_2)):
        if((combined_solution_2[el]==value_1_1 or combined_solution_2[el]==value_1_2 ) and ( value_2_1==None and value_2_2==None )):           
            value_2_1 = combined_solution_2[el]
            index2_1 = el
            print(" index " , index2_1 ," VALUE " ,value_2_1 )
        elif((combined_solution_2[el]==value_1_1 or combined_solution_2[el]==value_1_2 ) and ( value_2_1!=None and value_2_2==None )):           
            value_2_2 = combined_solution_2[el]  
            index2_2 = el
            print(" index " , index2_2 , " VALUE " ,value_2_2 )
    pieces_Gene = combined_solution_2[index2_1+1: index2_2]


    composition = [] 

    first = combined_solution_2[:index2_1]

    if(len(first)!=0 or first==None):
        composition.extend(first)
    composition.append(value_2_1)
    composition.append(value_2_2)

    print(composition)
    print("peices gene value " , pieces_Gene , type(pieces_Gene))
    if(len(pieces_Gene)!=0 ):
        pieces_Gene=pieces_Gene[::-1]
        composition.extend(pieces_Gene)
    print(composition)

    ####!!!!!! extract better this and ok 
    if index2_2 == len(combined_solution_2) - 1:
        last = []  # Return an empty list if the index corresponds to the last element
    else:
        last = combined_solution_2[index2_2 + 1:]  # Extract values from the last element to the specified
    composition.extend(last)
    print(composition)
    ##delete the last element after the combination add the start element at the end
    return "array"

In [209]:
list1 = [4, 7, 1, 3, 9, 6, 2, 8, 0, 5, 4]
list2 = [2, 8, 5, 1, 9, 0, 6, 3, 7, 4, 2] 

xo =inver_over_crossover( list1 , list2)
print(xo)


[4, 7, 1, 3, 9, 6, 2, 8, 0, 5, 4]
[2, 8, 5, 1, 9, 0, 6, 3, 7, 4, 2]
[4, 7, 1, 3, 9, 6, 2, 8, 0, 5]
[2, 8, 5, 1, 9, 0, 6, 3, 7, 4]  prendere da qui
1 element  7 1
 index  3  VALUE  1
 index  8  VALUE  7
[2, 8, 5, 1, 7]
peices gene value  [9, 0, 6, 3] <class 'list'>
[2, 8, 5, 1, 7, 3, 6, 0, 9]
[2, 8, 5, 1, 7, 3, 6, 0, 9, 4]
array


Order Crossover (OX)
• Try to preserve the relative order
• Partially Mapped Crossover (PMX)
• Try to preserve the relative order
• Cycle Crossover (CX)
• Combines cycles of elements from the parents

## TRAIN LOOP

In [20]:
#tsp             # this is the starting point 
#print(tsp)


# tabella soluzioni - index - object - cost

### generational APPROACH we REPLACE THE PARENTS , WE LOSING THE SOLUTION
 
population = []

off_string = []



print(len(DIST_MATRIX))

for index in range(len(DIST_MATRIX)):
    solution = GreedyAlgorithm(index)
    population.append(solution)

population = sorted(population ,  key= lambda x : x[1])
print(len(population))

INFO:root:result: Found a path of 46 steps, total length 4436.03km
INFO:root:result: Found a path of 46 steps, total length 5193.69km
INFO:root:result: Found a path of 46 steps, total length 5015.50km
INFO:root:result: Found a path of 46 steps, total length 4737.44km
INFO:root:result: Found a path of 46 steps, total length 4857.08km
INFO:root:result: Found a path of 46 steps, total length 5164.96km
INFO:root:result: Found a path of 46 steps, total length 4660.15km
INFO:root:result: Found a path of 46 steps, total length 5254.72km
INFO:root:result: Found a path of 46 steps, total length 5128.69km
INFO:root:result: Found a path of 46 steps, total length 4872.88km
INFO:root:result: Found a path of 46 steps, total length 4760.19km
INFO:root:result: Found a path of 46 steps, total length 5122.18km
INFO:root:result: Found a path of 46 steps, total length 5040.86km
INFO:root:result: Found a path of 46 steps, total length 4761.34km
INFO:root:result: Found a path of 46 steps, total length 4898.

46
46


- create solution using the greedy algorithm but using a different starting point

## Second Greedy Algorithm -NOT YET IMPLEMENTED BY THE PROFESSOR

NOT YET IMPLEMENTED BY THE PROFESSOR

In [20]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
]
visited = set()
edges = set()

shortest = next(_ for _ in sorted(segments, key=lambda e: e[1]))
visited |= shortest[0]
print(visited)
print(shortest)

edges |= {tuple(shortest[0])}
segments = [s for s in segments if not cyclic(edges | {tuple(s[0])})]

print(edges)
print(segments)

print(cyclic(edges))

print(shortest)

## ALGORITHM TO FIND SOLUTION


working but worst then the greedy maybe depending from the starting
greedy algorithm uses different starting point

new algorithm modification of the greedy but withsome modification to avoid ping_pong effect
new starting point

In [ ]:
visited = np.full(len(CITIES), False)
dist = segments.copy()
#print(dist)
iterator = iter(sorted(shortest[0]))
city = next(iterator)
#print(city)
next_city = next(iterator)
#print(next_city)
visited[city] = True
tsp = list()
tsp.append(int(city))

previous_point= shortest
#print("first point " , previous_point)
i=0
logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[next_city,'name']} ({DIST_MATRIX[city,next_city]:.2f}km)"
    )
#for i in range(4):
while not np.all(visited):
    #print()
    #print(i) 
    #print(previous_point)

    dist = [ distance for distance in dist if not all(previous_point[0] in distance for item in dist) ] ### remove previous point 
    
    pick_next_city = []
    for edge in dist:
        tuple_set= edge[0]
        if(next_city in tuple_set ):              ###### we save the tuple in which there are the next city x -> y
            pick_next_city.append(edge)           ###### filtering only the points with the next city 

    pick_next_city=sorted(pick_next_city, key=lambda x: x[1])
    #print(pick_next_city)
    if(len(pick_next_city)==0):
        visited[next_city] = True
        tsp.append(int(next_city))
        #print(visited)
        break
    
    boolean_flag=False
    for element in pick_next_city:
        #print()
        tupla = element[0]
        distance= element[1]    
        #print("element printed " , element)
        filtered_values = {value for value in tupla if value != next_city}.pop()  ### we are extracting the value from the set that are not included
                                                                                  ###    (y , z) we extract z 
        extract = [ distance for distance in dist if all( {filtered_values, city} in distance for item in dist  )  ].pop() # distance STARTING POINT - NEW POINT WITHOUT MIDDLE
                                                                                                                            ### we save the tuple (x, z)
        #print(filtered_values , "   " , city)
        #print("extracted mean (x, z)" , extract)
        #print("comparison between " , previous_point, extract)

        if(extract[1]>previous_point[1]):             #### comparison of distance of  ( x, z ) > (x , y )
            boolean_flag=True
            dist = [ distance for distance in dist if city not in distance[0]   ]      #clean the distance- we are removing all starting point

            #print("the length of dist is ",len(dist))
            previous_point= element
            #print(previous_point)
            #print("the new point is " ,previous_point)
            city= next_city
            #print("the city is ", city)

            next_city =  filtered_values
            #print("the next is",next_city)
            break
        if(boolean_flag==False):
            print("ERROR LOOP IN")
            #print(city , next_city)
    #i+=1
    visited[next_city] = True
    tsp.append(int(city))
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[next_city,'name']} ({DIST_MATRIX[city,next_city]:.2f}km)"
    )

logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
    )
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

continue the exercise using the mutation and cross_over

## ALGORITHM with WORST SOLUTION

From the solution - apply a Local Search 

In [ ]:
visited = np.full(len(CITIES), False)
G = nx.Graph()
dist = segments.copy()
city = next(iter(sorted(shortest[0])))
visited[city] = True

tsp = list()
tsp.append(int(city))


while not np.all(visited):
#for i in range(1):
    ### here we decide how to pick the next 
    picking_next = [  tup  for tup in dist if city in tup[0]  ]
    #print( picking_next )
    ### computing some statistical measure
    distances = [tup[1]  for tup in dist  ]
    mean = np.mean(distances)
    variance = np.var(distances, ddof=0) 

    rank_obj = []
    for index_value in  picking_next:
        tuple_index = index_value[0]
        distance= index_value[1]
        if( distance > mean):
            rank_obj.append(index_value)
    
    if(len(rank_obj)!=0):
        rank_obj=sorted(rank_obj, key=lambda x: x[1])
        next_city = rank_obj[0]
    else:
        if(len(picking_next)!=1):
            for index_value in  picking_next:
                tuple_index = index_value[0]
                distance= index_value[1]
                print(distance , mean)
                if( distance < mean):
                    rank_obj.append(index_value)
            rank_obj = sorted(rank_obj, key=lambda x: x[1])
            if(len(rank_obj)!=1):
                next_city = rank_obj[0]
        else:
            next_city = picking_next[0]

    for indexis in next_city[0]:
        if(indexis!=city):
            feet= indexis
    dist = [  tup  for tup in dist if city not in tup[0]       ] 
    #closest = np.argmin(dist[city])    #### here we are finding the closest city 


    G.add_nodes_from([city, feet])
    G.add_edge(city, feet, weight=round(next_city[1], 2) )



    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[feet,'name']} ({DIST_MATRIX[city,feet]:.2f}km)"
    )
    visited[feet] = True
    city = feet
    tsp.append(int(city))


logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")    # assertion error

continue the algorithm using the algorithm

## General Graph - Representation

In [67]:
# Create an undirected graph
G = nx.Graph()

for element in segments:
    print()
    print(element)
    # Add nodes
    iterator = iter(element[0])
    A=next(iterator)
    B=next(iterator)    
    G.add_nodes_from([A, B])
    # Add an undirected edge with a weight

    G.add_edge(A, B, weight=round(element[1], 2) )

# Check the edge with weight
#print("Edges with weights:", G.edges(data=True))
# Define the position of nodes for a simple layout
pos = nx.spring_layout(G)


# Increase figure size
plt.figure(figsize=(50, 60))  # Width=10, Height=6
# Draw the nodes and edges
nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=500, font_size=15)

# Draw edge labels (weights)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Display the graph
plt.show()

NameError: name 'segments' is not defined